In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
import pytorch_ssim
from net.model import SS_UIE_model
from utils.utils import *
from utils.LAB import *
from utils.LCH import *
from utils.FDL import *
import cv2
import time as time
import datetime
import sys
from torchvision.utils import save_image
import csv
import random
import torch.utils.data as dataf
import torch.nn.functional as F

/home/plt/miniconda3/envs/mamba/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
dtype = 'float32'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
torch.cuda.set_device(0)
torch.set_default_tensor_type(torch.FloatTensor)

/home/plt/miniconda3/envs/mamba/lib/python3.10/site-packages/torch/__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /opt/conda/conda-bld/pytorch_1699449181202/work/torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)


In [ ]:
# Initialize generator 
SS_UIE = SS_UIE_model(in_channels=3,  channels=16, num_resblock=4, num_memblock=4).cuda()
SS_UIE = nn.DataParallel(SS_UIE)
SS_UIE = SS_UIE.to('cuda:0')
SS_UIE.load_state_dict(torch.load("./saved_models/SS_UIE.pth"))

<All keys matched successfully>

In [4]:
SS_UIE.eval()

DataParallel(
  (module): SS_UIE_model(
    (extra_conv1): BNReLUConv(
      (bn): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (extra_conv2): BNReLUConv(
      (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (extra_conv3): BNReLUConv(
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (recons_conv1): BNReLUConv(
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplac

In [5]:
path='./data/Test_400/input/'#要改
path_list = os.listdir(path)
path_list.sort(key=lambda x:int(x.split('.')[0]))

for item in path_list:
    impath=path+item
    imgx= cv2.imread(path+item)
    imgx=cv2.resize(imgx,(256,256))
    imgx = cv2.cvtColor(imgx, cv2.COLOR_BGR2RGB)
    imgx = np.array(imgx).astype(dtype)

    imgx= torch.from_numpy(imgx)
    imgx=imgx.permute(2,0,1).unsqueeze(0)
    imgx=imgx/255.0
    #plt.imshow(imgx[0,:,:,:])
    #plt.show()
    imgx = Variable(imgx).cuda()
    #print(imgx.shape)
    out=SS_UIE(imgx)
    img_name="./data/Test_400/output/"+item
    save_image(out, img_name, nrow=5, normalize=True)

